In [22]:
# Using create_project_summary.py as a reference, grab the list of all project from survey admin
# Take an input from the user which limits to projects with a Start Date of in the past X period of time
# For each project in that timeframe, download the CSV
# move the CSVs to a directory

In [23]:
import se_admin, se_general
from config import Config
import bs4, os, subprocess, datetime, shutil, time, sys, openpyxl
import pandas as pd
from pprint import pprint
import pyautogui
import gui
import win32com.client as win32

cfg = Config()

In [24]:
# scrape the latest survey admin table

driver = se_general.init_selenium()
driver.implicitly_wait(30)
live_jobs, all_jobs = se_admin.grab_sa_projects_dicts(driver)

 2023-05-31 19:53:48,747 - INFO - Applicable driver not found; attempting to install with Selenium Manager (Beta)
 2023-05-31 19:53:48,748 - DEBUG - Executing process: C:\Users\kevin\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\common\windows\selenium-manager.exe --browser chrome --output json --debug
 2023-05-31 19:53:48,918 - DEBUG - Using shell command to find out chrome version
 2023-05-31 19:53:48,918 - DEBUG - Running command: "wmic datafile where name='%PROGRAMFILES:\\=\\\\%\\\\Google\\\\Chrome\\\\Application\\\\chrome.exe' get Version /value"
 2023-05-31 19:53:48,918 - DEBUG - Output: "\r\r\n\r\r\nVersion=113.0.5672.127\r\r\n\r\r\n\r\r\n\r"
 2023-05-31 19:53:48,919 - DEBUG - The version of chrome is 113.0.5672.127
 2023-05-31 19:53:48,919 - DEBUG - Detected browser: chrome 113
 2023-05-31 19:53:48,919 - DEBUG - Required driver: chromedriver 113.0.5672.63
 2023-05-31 19:53:48,919 - DEBUG - Running command: "chromedriver --version"
 2023-05-31 19:5

running login_sa_2fa function


 2023-05-31 19:53:50,423 - DEBUG - http://localhost:50479 "POST /session/330a02588ec749e46ddb6411501982b8/url HTTP/1.1" 200 14
 2023-05-31 19:53:50,424 - DEBUG - Remote response: status=200 | data={"value":null} | headers=HTTPHeaderDict({'Content-Length': '14', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
 2023-05-31 19:53:50,424 - DEBUG - Finished Request
 2023-05-31 19:53:50,425 - DEBUG - POST http://localhost:50479/session/330a02588ec749e46ddb6411501982b8/element {"using": "css selector", "value": "[id=\"i0116\"]"}
 2023-05-31 19:53:50,500 - DEBUG - http://localhost:50479 "POST /session/330a02588ec749e46ddb6411501982b8/element HTTP/1.1" 200 94
 2023-05-31 19:53:50,501 - DEBUG - Remote response: status=200 | data={"value":{"element-6066-11e4-a52e-4f735466cecf":"8777C61268669AE6DF2278A34A204BE7_element_3"}} | headers=HTTPHeaderDict({'Content-Length': '94', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
 2023-05-31 1

Attempting to load URL: https://data.studentedge.org/admin/survey/index


 2023-05-31 19:53:51,542 - DEBUG - POST http://localhost:50479/session/330a02588ec749e46ddb6411501982b8/element {"using": "css selector", "value": "[id=\"idSIButton9\"]"}
 2023-05-31 19:53:51,549 - DEBUG - http://localhost:50479 "POST /session/330a02588ec749e46ddb6411501982b8/element HTTP/1.1" 200 95
 2023-05-31 19:53:51,550 - DEBUG - Remote response: status=200 | data={"value":{"element-6066-11e4-a52e-4f735466cecf":"8777C61268669AE6DF2278A34A204BE7_element_16"}} | headers=HTTPHeaderDict({'Content-Length': '95', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
 2023-05-31 19:53:51,550 - DEBUG - Finished Request
 2023-05-31 19:53:51,551 - DEBUG - POST http://localhost:50479/session/330a02588ec749e46ddb6411501982b8/element/8777C61268669AE6DF2278A34A204BE7_element_16/click {"id": "8777C61268669AE6DF2278A34A204BE7_element_16"}
 2023-05-31 19:53:51,573 - DEBUG - http://localhost:50479 "POST /session/330a02588ec749e46ddb6411501982b8/element/8777C61268669AE6DF2

Running export_soup()


C:\Github local repos\se_internal\se_admin.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  live_jobs_df.rename(columns={'Unnamed: 1': 'Survey ID'}, inplace=True)  # change weird naming of survey_id col


head of sa_jobs_df looks like this:
                             Name                             Survey ID   
0  ISA Tracker 2022-2023 - Jun-23  c133b54d-261f-44cc-b71d-b01201564af0  \
1                  QLD sun safety  b8833dc5-e315-4e2f-a995-b013002b4db6   
2   Consumer Atlas Jun-23 (14-17)  c41f2b22-44d5-49b3-84f9-b00700329155   
3   Consumer Atlas Jun-23 (25-29)  cd15a437-dac6-4edf-986e-b007003563f4   
4                   Citizen 14-24  d5985746-eaa0-450e-b9c1-b00e0070cdc7   

                       Title Project/IO Number               Client Company   
0               Super Survey           P-48901             Research Profile  \
1      QLD sun safety survey           P-48902                      The ORU   
2  Money for 14-17 year olds           P-48884  Retail Finance Intelligence   
3  Money for 25-29 year olds           P-48885  Retail Finance Intelligence   
4               Youth Survey           P-48896                         Cint   

   Status  Start Date    End Date Expe

In [25]:
# save the all_jobs dictionary to a local json file in the private\json folder

se_general.save_to_json(cfg.json_path / 'all_jobs.json', all_jobs)
se_general.save_to_json(cfg.json_path / 'live_jobs.json', live_jobs)

In [26]:
live_jobs = se_general.load_from_json(cfg.json_path / 'all_jobs.json')

In [27]:
live_jobs

{'ISA Tracker 2022-2023 - Jun-23': {'survey_name': 'ISA Tracker 2022-2023 - Jun-23',
  'survey_id': 'c133b54d-261f-44cc-b71d-b01201564af0',
  'topic': 'Super Survey',
  'p_number': 'P-48901',
  'client_name': 'Research Profile',
  'status': 'Active',
  'start_date': '2023/06/05',
  'end_date': '2024/08/31',
  'expected_loi': '12',
  'actual_loi': nan,
  'completes': 0,
  'screenouts': 0,
  'quota_fulls': 0,
  'published': '0%'},
 'QLD sun safety': {'survey_name': 'QLD sun safety',
  'survey_id': 'b8833dc5-e315-4e2f-a995-b013002b4db6',
  'topic': 'QLD sun safety survey',
  'p_number': 'P-48902',
  'client_name': 'The ORU',
  'status': 'Active',
  'start_date': '2023/05/31',
  'end_date': '2024/08/31',
  'expected_loi': '15',
  'actual_loi': 12.0,
  'completes': 5,
  'screenouts': 1,
  'quota_fulls': 0,
  'published': '83%'},
 'Consumer Atlas Jun-23 (14-17)': {'survey_name': 'Consumer Atlas Jun-23 (14-17)',
  'survey_id': 'c41f2b22-44d5-49b3-84f9-b00700329155',
  'topic': 'Money for 14-1

In [28]:
# create a pandas data frame from the all_jobs dictionary

df = pd.DataFrame(all_jobs)

# display the first 10 rows of the data frame

df.head(10)

,ISA Tracker 2022-2023 - Jun-23,QLD sun safety,Consumer Atlas Jun-23 (14-17),Consumer Atlas Jun-23 (25-29),Citizen 14-24,Survey of young Victorians 2023,Macquarie Brand Health Tracking Dip 13,MLT Tracker May-23 - Main,Content Creator Survey,Banking Tracker - 14-24 (Jun-23),...,KP demo job Nov-19,KP test 123 today,Cancelled (Music Jul-19),KP_test_22-10-20,KP 11-06-20 test 1,Deleted,Business development survey - cancelled,DV Campaign (b),STEM qual recruit,Thred Recruitment
survey_name,ISA Tracker 2022-2023 - Jun-23,QLD sun safety,Consumer Atlas Jun-23 (14-17),Consumer Atlas Jun-23 (25-29),Citizen 14-24,Survey of young Victorians 2023,Macquarie Brand Health Tracking Dip 13,MLT Tracker May-23 - Main,Content Creator Survey,Banking Tracker - 14-24 (Jun-23),...,KP demo job Nov-19,KP test 123 today,Cancelled (Music Jul-19),KP_test_22-10-20,KP 11-06-20 test 1,Deleted,Business development survey - cancelled,DV Campaign (b),STEM qual recruit,Thred Recruitment
survey_id,c133b54d-261f-44cc-b71d-b01201564af0,b8833dc5-e315-4e2f-a995-b013002b4db6,c41f2b22-44d5-49b3-84f9-b00700329155,cd15a437-dac6-4edf-986e-b007003563f4,d5985746-eaa0-450e-b9c1-b00e0070cdc7,3d4be00d-7770-458d-8e08-b00e0025a272,32509ad8-e1ab-4302-898e-b00d00144787,eb8bbedf-a7f9-4e39-949e-b00b004dff23,1471054a-c11a-4f46-b987-b00b005ff999,9b00231f-e778-4f8b-a41c-b00a0047b0cc,...,cfb86553-269f-479f-bcb5-ab1200394c70,cea8ee63-f162-4fba-8dc4-ab12005f4022,9c083582-5630-488e-8313-aa8f007bfb57,c3f23722-7589-4a4f-83b7-ac5c00b24579,356ffff5-44bc-492d-89df-abd700407c41,44126eb3-d4a6-4584-b885-af0200285a9d,f2e87cc6-63e9-4dbd-8dcd-af02001d50ca,bb1709d5-8aa9-4c05-b6a8-a751006cbc73,0855ea43-9101-41e3-a78c-a81100411956,441f0f03-bee8-4256-9b70-a7240047eb28
topic,Super Survey,QLD sun safety survey,Money for 14-17 year olds,Money for 25-29 year olds,Youth Survey,Victoria Youth Survey,Macquarie Uni Survey,Sports Survey,Become a paid content creator with Student Edg...,Banking Survey,...,Test,topic stuff,Cancelled,test topic,test,Deleted,Online survey for 14-26 year olds,Advertising Survey,STEM qual recruit,Qual Recruitment Survey
p_number,P-48901,P-48902,P-48884,P-48885,P-48896,P-48895,P-48894,P-48889,P-48890,P-48887,...,P-46849,P-46858,Cancelled,P-47341,test,Deleted,P-48454,P-45061,tbc,P-44942
client_name,Research Profile,The ORU,Retail Finance Intelligence,Retail Finance Intelligence,Cint,Survey of young Victorians 2023,IPSOS Pty Ltd,Dynata Global AU Pty Ltd,YouthInsight Internal,Fifth Dimension,...,YouthInsight Internal,YouthInsight Internal,Cancelled,Dynata Global AU Pty Ltd,test,Deleted,YouthInsight Internal,Kantar Public,Telmy,Thred
status,Active,Active,Active,Active,Active,Active,Active,Active,Active,Active,...,Closed,Closed,Closed,Active,Pending,Closed,Active,Closed,Closed,Closed
start_date,2023/06/05,2023/05/31,2023/05/29,2023/05/29,2023/05/26,2023/05/26,2023/05/25,2023/05/23,2023/05/23,2023/05/22,...,2013/11/27,2013/11/27,2013/07/19,2012/10/22,2012/06/11,2010/09/01,2001/09/01,NaN,NaN,NaN
end_date,2024/08/31,2024/08/31,2024/08/31,2024/08/31,2024/08/31,2024/08/31,2024/08/31,2024/08/31,2024/08/31,2024/08/31,...,2013/12/31,2013/11/28,2013/08/31,2012/11/30,2012/07/31,2010/10/31,2001/10/31,NaN,NaN,NaN
expected_loi,12,15,28,37,10,8,15,27,4,24,...,20,10,16,4,5,Deleted,5,22,5,3
actual_loi,NaN,12.0,28.0,229.0,9.0,8.0,NaN,27.0,4.0,24.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [29]:
# swap columns and rows
df = df.T

In [30]:
# remove the index
df.reset_index(inplace=True)

In [31]:
df.head()

,index,survey_name,survey_id,topic,p_number,client_name,status,start_date,end_date,expected_loi,actual_loi,completes,screenouts,quota_fulls,published
0,ISA Tracker 2022-2023 - Jun-23,ISA Tracker 2022-2023 - Jun-23,c133b54d-261f-44cc-b71d-b01201564af0,Super Survey,P-48901,Research Profile,Active,2023/06/05,2024/08/31,12,NaN,0,0,0,0%
1,QLD sun safety,QLD sun safety,b8833dc5-e315-4e2f-a995-b013002b4db6,QLD sun safety survey,P-48902,The ORU,Active,2023/05/31,2024/08/31,15,12.0,5,1,0,83%
2,Consumer Atlas Jun-23 (14-17),Consumer Atlas Jun-23 (14-17),c41f2b22-44d5-49b3-84f9-b00700329155,Money for 14-17 year olds,P-48884,Retail Finance Intelligence,Active,2023/05/29,2024/08/31,28,28.0,16,8,0,67%
3,Consumer Atlas Jun-23 (25-29),Consumer Atlas Jun-23 (25-29),cd15a437-dac6-4edf-986e-b007003563f4,Money for 25-29 year olds,P-48885,Retail Finance Intelligence,Active,2023/05/29,2024/08/31,37,229.0,7,4,0,64%
4,Citizen 14-24,Citizen 14-24,d5985746-eaa0-450e-b9c1-b00e0070cdc7,Youth Survey,P-48896,Cint,Active,2023/05/26,2024/08/31,10,9.0,98,63,9,61%


In [32]:
# delete the column in df named index
del df['index']

In [33]:
import pandas as pd
from datetime import datetime, timedelta

# Convert 'start_date' column to datetime if it's not already in the datetime format
df['start_date'] = pd.to_datetime(df['start_date'])

# Get the current date
current_date = datetime.now()

# Calculate the date 30 days ago
past_30_days = current_date - timedelta(days=30)

# Apply the filter to include only rows where 'start_date' is within the past 30 days
last_30_days_df = df[df['start_date'] >= past_30_days]


In [34]:
last_30_days_df

,survey_name,survey_id,topic,p_number,client_name,status,start_date,end_date,expected_loi,actual_loi,completes,screenouts,quota_fulls,published
0,ISA Tracker 2022-2023 - Jun-23,c133b54d-261f-44cc-b71d-b01201564af0,Super Survey,P-48901,Research Profile,Active,2023-06-05,2024/08/31,12,NaN,0,0,0,0%
1,QLD sun safety,b8833dc5-e315-4e2f-a995-b013002b4db6,QLD sun safety survey,P-48902,The ORU,Active,2023-05-31,2024/08/31,15,12.0,5,1,0,83%
2,Consumer Atlas Jun-23 (14-17),c41f2b22-44d5-49b3-84f9-b00700329155,Money for 14-17 year olds,P-48884,Retail Finance Intelligence,Active,2023-05-29,2024/08/31,28,28.0,16,8,0,67%
3,Consumer Atlas Jun-23 (25-29),cd15a437-dac6-4edf-986e-b007003563f4,Money for 25-29 year olds,P-48885,Retail Finance Intelligence,Active,2023-05-29,2024/08/31,37,229.0,7,4,0,64%
4,Citizen 14-24,d5985746-eaa0-450e-b9c1-b00e0070cdc7,Youth Survey,P-48896,Cint,Active,2023-05-26,2024/08/31,10,9.0,98,63,9,61%
5,Survey of young Victorians 2023,3d4be00d-7770-458d-8e08-b00e0025a272,Victoria Youth Survey,P-48895,Survey of young Victorians 2023,Active,2023-05-26,2024/08/31,8,8.0,36,4,0,90%
6,Macquarie Brand Health Tracking Dip 13,32509ad8-e1ab-4302-898e-b00d00144787,Macquarie Uni Survey,P-48894,IPSOS Pty Ltd,Active,2023-05-25,2024/08/31,15,NaN,0,0,0,0%
7,MLT Tracker May-23 - Main,eb8bbedf-a7f9-4e39-949e-b00b004dff23,Sports Survey,P-48889,Dynata Global AU Pty Ltd,Active,2023-05-23,2024/08/31,27,27.0,72,46,0,61%
8,Content Creator Survey,1471054a-c11a-4f46-b987-b00b005ff999,Become a paid content creator with Student Edg...,P-48890,YouthInsight Internal,Active,2023-05-23,2024/08/31,4,4.0,63,517,0,11%
9,Banking Tracker - 14-24 (Jun-23),9b00231f-e778-4f8b-a41c-b00a0047b0cc,Banking Survey,P-48887,Fifth Dimension,Active,2023-05-22,2024/08/31,24,24.0,66,77,0,46%


In [35]:
nov_2022_df = df[(df['start_date'].dt.month == 11) & (df['start_date'].dt.year == 2022)]


In [36]:
nov_2022_df

,survey_name,survey_id,topic,p_number,client_name,status,start_date,end_date,expected_loi,actual_loi,completes,screenouts,quota_fulls,published
148,Coles Tracker - 18-24s (Dec-22),a9190f58-be73-4dda-aaa3-af56017b2197,Supermarket Survey,P-48606,Fifth Dimension,Active,2022-11-30,2024/01/31,17,17.0,258,230,1,53%
149,MLT Tracker Nov-22 - Main,2d68108e-20c2-4372-ba0c-af5c0035af77,Sports Survey,P-48617,Dynata Global AU Pty Ltd,Active,2022-11-29,2024/02/28,22,21.0,149,85,66,64%
150,Banking Tracker - 14-24 (Dec-22),70a08659-9f70-4c6b-a5b3-af56017e9d91,Banking Survey for Males,P-48607,Fifth Dimension,Active,2022-11-29,2024/01/31,15,15.0,193,139,0,58%
151,WA Youngsters Dec-22,6895768c-dbbd-458e-b882-af5c000b1286,Mental Health Survey,P-48615,The ORU,Active,2022-11-29,2023/02/28,18,18.0,102,9,0,92%
152,Consumer Atlas Dec-22 (14-17),4af1fb76-a43e-493f-86a2-af50017d4818,Money for 14-17 year olds,P-48599,DBM Consultants,Active,2022-11-28,2024/01/31,12,12.0,198,34,18,85%
153,Consumer Atlas Dec-22 (25-29),165929dd-38e3-4e4a-bf4f-af500180c528,Money for 25-29 year olds,P-48600,DBM Consultants,Active,2022-11-28,2024/01/31,24,24.0,25,9,0,74%
154,Macquarie Brand Health Tracking Dip 12,73341c5f-fd1a-439f-ba2d-af57017394fc,Macquarie Uni Survey,P-48610,Iview Pty Ltd,Active,2022-11-25,2024/01/31,14,14.0,99,80,56,55%
155,FS Tracker - NSW Prosp UG Dec-22,4ff5e47c-1ef5-4327-bca1-af5701807ded,Uni Survey,P-48611,University of Technology (Sydney),Active,2022-11-25,2024/01/31,7,7.0,283,397,0,42%
156,University Brand Study - Dec22,1143b8a2-042f-4bfb-b2d1-af5700503ebc,Considering Uni survey for 14-19 year olds,P-48608,Dynata Global AU Pty Ltd,Active,2022-11-24,2024/01/31,11,11.0,107,83,61,56%
157,Griffith Dec-22,014fe8db-4379-4237-bcea-af56003ca2de,Uni & Future Uni Students,P-48604,I-Link Research Solutions Pty Ltd,Active,2022-11-23,2024/01/31,17,17.0,56,40,33,58%


In [37]:
# put each value from the survey_id field in nov_2022_df into a python list
survey_ids = nov_2022_df['survey_id'].tolist()

In [38]:
survey_ids

['a9190f58-be73-4dda-aaa3-af56017b2197',
 '2d68108e-20c2-4372-ba0c-af5c0035af77',
 '70a08659-9f70-4c6b-a5b3-af56017e9d91',
 '6895768c-dbbd-458e-b882-af5c000b1286',
 '4af1fb76-a43e-493f-86a2-af50017d4818',
 '165929dd-38e3-4e4a-bf4f-af500180c528',
 '73341c5f-fd1a-439f-ba2d-af57017394fc',
 '4ff5e47c-1ef5-4327-bca1-af5701807ded',
 '1143b8a2-042f-4bfb-b2d1-af5700503ebc',
 '014fe8db-4379-4237-bcea-af56003ca2de',
 'eef1078e-634f-46bb-ac4f-af50000a7d5a',
 '94859a09-015a-485c-aaf2-af4e00713311',
 '58c1169b-4912-4f93-8dfd-af4e007735ce',
 '1f8fbd76-63f1-44be-8831-af4e003034de',
 'b24c679c-0d0f-4024-9d48-af49017b3a13',
 '6a6589db-ec6e-40b9-8367-af4701875df1',
 '07e5d86f-4ad8-401e-88d8-af48001f5a3a',
 '552f8497-d241-4f6b-8714-af470061465b',
 '124f0bf7-6e21-46f3-8a73-af46000cdd32',
 '7e082dbe-7e1b-4148-a25b-af42016c5cfc',
 '299355d0-9124-497f-976c-af4300189450',
 '8403f9e8-a714-414a-af4f-af4300544ac9',
 'c366e0cc-19d4-4cba-9cca-af42005efe9f',
 'd9a3dba3-3e2e-44c4-a021-af42005f903d',
 '237b139d-a054-

In [39]:
len(survey_ids)

36

In [40]:
# Download current results for each project
number_of_files_to_download = len(survey_ids)
count = 1
for survey_id in survey_ids:
    dl_link = f"{cfg.current_results_dl_url_prefix}{survey_id}"
    print(f'\nNow downloading file {count} of {number_of_files_to_download}')
    print(f'\nNow downloading SurveyResults for {dl_link}')
    driver.get(dl_link)  # commented out during test mode
    count += 1

 2023-05-31 19:54:15,933 - DEBUG - POST http://localhost:50479/session/330a02588ec749e46ddb6411501982b8/url {"url": "https://data.studentedge.org/admin/survey/download?id=a9190f58-be73-4dda-aaa3-af56017b2197"}



Now downloading file 1 of 36

Now downloading SurveyResults for https://data.studentedge.org/admin/survey/download?id=a9190f58-be73-4dda-aaa3-af56017b2197


 2023-05-31 19:54:48,693 - DEBUG - http://localhost:50479 "POST /session/330a02588ec749e46ddb6411501982b8/url HTTP/1.1" 200 14
 2023-05-31 19:54:48,694 - DEBUG - Remote response: status=200 | data={"value":null} | headers=HTTPHeaderDict({'Content-Length': '14', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
 2023-05-31 19:54:48,694 - DEBUG - Finished Request
 2023-05-31 19:54:48,694 - DEBUG - POST http://localhost:50479/session/330a02588ec749e46ddb6411501982b8/url {"url": "https://data.studentedge.org/admin/survey/download?id=2d68108e-20c2-4372-ba0c-af5c0035af77"}



Now downloading file 2 of 36

Now downloading SurveyResults for https://data.studentedge.org/admin/survey/download?id=2d68108e-20c2-4372-ba0c-af5c0035af77


 2023-05-31 19:55:15,342 - DEBUG - http://localhost:50479 "POST /session/330a02588ec749e46ddb6411501982b8/url HTTP/1.1" 200 14
 2023-05-31 19:55:15,343 - DEBUG - Remote response: status=200 | data={"value":null} | headers=HTTPHeaderDict({'Content-Length': '14', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
 2023-05-31 19:55:15,343 - DEBUG - Finished Request
 2023-05-31 19:55:15,343 - DEBUG - POST http://localhost:50479/session/330a02588ec749e46ddb6411501982b8/url {"url": "https://data.studentedge.org/admin/survey/download?id=70a08659-9f70-4c6b-a5b3-af56017e9d91"}


...

 2023-05-31 20:09:34,538 - DEBUG - http://localhost:50479 "POST /session/330a02588ec749e46ddb6411501982b8/url HTTP/1.1" 200 14
 2023-05-31 20:09:34,538 - DEBUG - Remote response: status=200 | data={"value":null} | headers=HTTPHeaderDict({'Content-Length': '14', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
 2023-05-31 20:09:34,538 - DEBUG - Finished Request
 2023-05-31 20:09:34,539 - DEBUG - POST http://localhost:50479/session/330a02588ec749e46ddb6411501982b8/url {"url": "https://data.studentedge.org/admin/survey/download?id=51f2de04-a651-432a-8681-af3e0171e735"}



Now downloading file 35 of 36

Now downloading SurveyResults for https://data.studentedge.org/admin/survey/download?id=51f2de04-a651-432a-8681-af3e0171e735


 2023-05-31 20:10:01,707 - DEBUG - http://localhost:50479 "POST /session/330a02588ec749e46ddb6411501982b8/url HTTP/1.1" 200 14
 2023-05-31 20:10:01,707 - DEBUG - Remote response: status=200 | data={"value":null} | headers=HTTPHeaderDict({'Content-Length': '14', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
 2023-05-31 20:10:01,708 - DEBUG - Finished Request
 2023-05-31 20:10:01,708 - DEBUG - POST http://localhost:50479/session/330a02588ec749e46ddb6411501982b8/url {"url": "https://data.studentedge.org/admin/survey/download?id=87cad387-79ce-4764-90fc-af3f00399582"}



Now downloading file 36 of 36

Now downloading SurveyResults for https://data.studentedge.org/admin/survey/download?id=87cad387-79ce-4764-90fc-af3f00399582


 2023-05-31 20:10:25,571 - DEBUG - http://localhost:50479 "POST /session/330a02588ec749e46ddb6411501982b8/url HTTP/1.1" 200 14
 2023-05-31 20:10:25,572 - DEBUG - Remote response: status=200 | data={"value":null} | headers=HTTPHeaderDict({'Content-Length': '14', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
 2023-05-31 20:10:25,572 - DEBUG - Finished Request


In [41]:
limited_df = df[(df['start_date'].dt.month == 10) & (df['start_date'].dt.year == 2022)]

# put each value from the survey_id field into a python list
survey_ids = limited_df['survey_id'].tolist()

limited_df

,survey_name,survey_id,topic,p_number,client_name,status,start_date,end_date,expected_loi,actual_loi,completes,screenouts,quota_fulls,published
185,Prospective UG on campus - Nov22,9585acc9-14f4-41a8-8e14-af3f00427b45,Education survey for year 12 students,P-48554,I-Link Research Solutions Pty Ltd,Active,2022-10-31,2023/12/31,10,10.0,180,31,0,85%
186,Finance Survey - Nov22,6f213369-730c-48d6-b323-af3f004a38b7,Finance Survey,P-48556,YouthInsight Internal,Active,2022-10-31,2023/12/31,6,6.0,516,19,0,96%
187,Consumer Atlas Nov-22 (14-17),c2b9192c-62eb-4a1d-b2e1-af3201806425,Money for 14-17 year olds,P-48535,DBM Consultants,Active,2022-10-26,2023/12/31,13,13.0,201,38,10,84%
188,Consumer Atlas Nov-22 (25-29),32d4af06-5e5d-4575-bef1-af32018334d9,Money for 25-29 year olds,P-48536,DBM Consultants,Active,2022-10-26,2022/12/31,26,27.0,23,32,0,42%
189,MOVE T&L Oct-22,f554fea3-65f8-4f16-89f7-af3a0027c6ba,Take part in series of activities and earn up ...,P-48545,IPSOS Australia,Active,2022-10-26,2022/12/31,2,2.0,570,328,0,63%
190,UQ Ad Effectiveness Oct-22,15653aac-0275-43ce-8e1d-af39005c190b,Considering Uni Survey,P-48543,Lightspeed Pty Ltd,Active,2022-10-25,2023/12/31,10,10.0,30,53,1,36%
191,MLT Tracker Oct-22 - Main,8ca2d964-dd29-423b-a7cc-af35002521f7,Sports Survey,P-48540,Dynata Global AU Pty Ltd,Active,2022-10-21,2023/12/31,20,20.0,163,65,3,71%
192,TV recruit Q2,08760777-2f29-40e6-a174-af33016ea319,Earn cash for talking about TV shows,P-48539,Insites Consulting,Active,2022-10-20,2022/12/31,5,6.0,28,22,4,56%
193,Establishment survey Oct-22,dac2ec08-0117-4318-90aa-af330063081a,Online survey,P-48538,Iview Pty Ltd,Active,2022-10-19,2023/12/31,8,8.0,65,43,32,60%
194,Product Team Recruitment Screener,8ef25042-3094-4385-981f-af33002d7962,Take part in a 30 minute Zoom interview about ...,P-48537,YouthInsight Internal,Active,2022-10-19,2023/12/31,5,5.0,19,39,0,33%


In [42]:
# Download current results for each project
number_of_files_to_download = len(survey_ids)
count = 1
for survey_id in survey_ids:
    dl_link = f"{cfg.current_results_dl_url_prefix}{survey_id}"
    print(f'\nNow downloading file {count} of {number_of_files_to_download}')
    print(f'\nNow downloading SurveyResults for {dl_link}')
    driver.get(dl_link)  # commented out during test mode
    count += 1

 2023-05-31 20:16:11,561 - DEBUG - POST http://localhost:50479/session/330a02588ec749e46ddb6411501982b8/url {"url": "https://data.studentedge.org/admin/survey/download?id=9585acc9-14f4-41a8-8e14-af3f00427b45"}



Now downloading file 1 of 23

Now downloading SurveyResults for https://data.studentedge.org/admin/survey/download?id=9585acc9-14f4-41a8-8e14-af3f00427b45


 2023-05-31 20:16:37,637 - DEBUG - http://localhost:50479 "POST /session/330a02588ec749e46ddb6411501982b8/url HTTP/1.1" 200 14
 2023-05-31 20:16:37,637 - DEBUG - Remote response: status=200 | data={"value":null} | headers=HTTPHeaderDict({'Content-Length': '14', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
 2023-05-31 20:16:37,638 - DEBUG - Finished Request
 2023-05-31 20:16:37,638 - DEBUG - POST http://localhost:50479/session/330a02588ec749e46ddb6411501982b8/url {"url": "https://data.studentedge.org/admin/survey/download?id=6f213369-730c-48d6-b323-af3f004a38b7"}



Now downloading file 2 of 23

Now downloading SurveyResults for https://data.studentedge.org/admin/survey/download?id=6f213369-730c-48d6-b323-af3f004a38b7


 2023-05-31 20:17:07,005 - DEBUG - http://localhost:50479 "POST /session/330a02588ec749e46ddb6411501982b8/url HTTP/1.1" 200 14
 2023-05-31 20:17:07,005 - DEBUG - Remote response: status=200 | data={"value":null} | headers=HTTPHeaderDict({'Content-Length': '14', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
 2023-05-31 20:17:07,006 - DEBUG - Finished Request
 2023-05-31 20:17:07,006 - DEBUG - POST http://localhost:50479/session/330a02588ec749e46ddb6411501982b8/url {"url": "https://data.studentedge.org/admin/survey/download?id=c2b9192c-62eb-4a1d-b2e1-af3201806425"}


...

 2023-05-31 20:25:22,006 - DEBUG - http://localhost:50479 "POST /session/330a02588ec749e46ddb6411501982b8/url HTTP/1.1" 200 14
 2023-05-31 20:25:22,007 - DEBUG - Remote response: status=200 | data={"value":null} | headers=HTTPHeaderDict({'Content-Length': '14', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
 2023-05-31 20:25:22,007 - DEBUG - Finished Request
 2023-05-31 20:25:22,007 - DEBUG - POST http://localhost:50479/session/330a02588ec749e46ddb6411501982b8/url {"url": "https://data.studentedge.org/admin/survey/download?id=941aadb5-092c-4481-af7b-af1b018aa179"}



Now downloading file 22 of 23

Now downloading SurveyResults for https://data.studentedge.org/admin/survey/download?id=941aadb5-092c-4481-af7b-af1b018aa179


 2023-05-31 20:25:45,997 - DEBUG - http://localhost:50479 "POST /session/330a02588ec749e46ddb6411501982b8/url HTTP/1.1" 200 14
 2023-05-31 20:25:45,998 - DEBUG - Remote response: status=200 | data={"value":null} | headers=HTTPHeaderDict({'Content-Length': '14', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
 2023-05-31 20:25:45,998 - DEBUG - Finished Request
 2023-05-31 20:25:45,998 - DEBUG - POST http://localhost:50479/session/330a02588ec749e46ddb6411501982b8/url {"url": "https://data.studentedge.org/admin/survey/download?id=e6278939-ee4f-4bf9-b902-af10000062da"}



Now downloading file 23 of 23

Now downloading SurveyResults for https://data.studentedge.org/admin/survey/download?id=e6278939-ee4f-4bf9-b902-af10000062da


 2023-05-31 20:26:12,961 - DEBUG - http://localhost:50479 "POST /session/330a02588ec749e46ddb6411501982b8/url HTTP/1.1" 200 14
 2023-05-31 20:26:12,962 - DEBUG - Remote response: status=200 | data={"value":null} | headers=HTTPHeaderDict({'Content-Length': '14', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
 2023-05-31 20:26:12,962 - DEBUG - Finished Request


In [43]:
# Filter condition for start_date after 31st November 2022
filtered_df = df[df['start_date'] > pd.Timestamp('2022-11-30')]
filtered_df

,survey_name,survey_id,topic,p_number,client_name,status,start_date,end_date,expected_loi,actual_loi,completes,screenouts,quota_fulls,published
0,ISA Tracker 2022-2023 - Jun-23,c133b54d-261f-44cc-b71d-b01201564af0,Super Survey,P-48901,Research Profile,Active,2023-06-05,2024/08/31,12,NaN,0,0,0,0%
1,QLD sun safety,b8833dc5-e315-4e2f-a995-b013002b4db6,QLD sun safety survey,P-48902,The ORU,Active,2023-05-31,2024/08/31,15,12.0,5,1,0,83%
2,Consumer Atlas Jun-23 (14-17),c41f2b22-44d5-49b3-84f9-b00700329155,Money for 14-17 year olds,P-48884,Retail Finance Intelligence,Active,2023-05-29,2024/08/31,28,28.0,16,8,0,67%
3,Consumer Atlas Jun-23 (25-29),cd15a437-dac6-4edf-986e-b007003563f4,Money for 25-29 year olds,P-48885,Retail Finance Intelligence,Active,2023-05-29,2024/08/31,37,229.0,7,4,0,64%
4,Citizen 14-24,d5985746-eaa0-450e-b9c1-b00e0070cdc7,Youth Survey,P-48896,Cint,Active,2023-05-26,2024/08/31,10,9.0,98,63,9,61%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,Domestic Tracker 16-19 Dec-22,03a943c0-f502-498a-ae64-af62003967fa,Buying underwear for 16-19,P-48625,Pureprofile AU,Active,2022-12-05,2024/02/28,21,26.0,26,7,6,79%
144,CMV 2022 December Modular,f133d4e7-a3b5-4610-ad93-af5e015e8eb0,$5 to $14 survey,P-48622,Dynata Global AU Pty Ltd,Active,2022-12-05,2023/12/15,55,55.0,38,42,31,48%
145,NPS EDM Survey - Dec 2022,6ffc4c0c-736b-4290-a5c4-af5c0020483e,Give your feedback on your experience of Stude...,P-48616,YouthInsight Internal,Active,2022-12-02,2024/02/29,1,1.0,994,0,1,100%
146,Iris Recruit Nov-22 invoicing,3ef5e6b0-4c00-417f-83c1-af5f001ca572,$15-30 per month passive income,P-48623,IPSOS Australia,Active,2022-12-02,2023/02/28,5,NaN,0,0,0,0%


In [44]:
# put each value from the survey_id field into a python list
survey_ids = filtered_df['survey_id'].tolist()

survey_ids

['c133b54d-261f-44cc-b71d-b01201564af0',
 'b8833dc5-e315-4e2f-a995-b013002b4db6',
 'c41f2b22-44d5-49b3-84f9-b00700329155',
 'cd15a437-dac6-4edf-986e-b007003563f4',
 'd5985746-eaa0-450e-b9c1-b00e0070cdc7',
 '3d4be00d-7770-458d-8e08-b00e0025a272',
 '32509ad8-e1ab-4302-898e-b00d00144787',
 'eb8bbedf-a7f9-4e39-949e-b00b004dff23',
 '1471054a-c11a-4f46-b987-b00b005ff999',
 '9b00231f-e778-4f8b-a41c-b00a0047b0cc',
 'ac8165a6-5780-4ab9-bfb1-b00a00366e5f',
 '69c5b38d-3ef0-4a3f-9413-b005017922b8',
 'bd7e0e10-3705-4100-8617-b006001ca434',
 '85130fc4-06f6-4c67-816d-b003018423df',
 '2a15c247-d019-499a-93b2-b004005cca24',
 '762e5d4f-66b9-4f41-9203-b004004b058c',
 'b3de56dc-ff26-4b72-9397-b00300481c96',
 '2013854e-c330-4259-944d-afff0024c2dd',
 'fee34c84-9b0d-4d5d-9da0-afff0080a6b6',
 'dac786dd-c67d-4ca7-a5df-affe0067e02a',
 '3f051f86-579c-4f97-b0e8-affe008f6f5d',
 '7018e17b-2903-43d4-a233-affd002cfc90',
 '7ef9bb71-3e05-4676-be13-affd000b37ad',
 '405ea710-db8c-4d1b-a05a-affd0026bfb3',
 'e00badf9-356d-

In [45]:
len(survey_ids)

147

In [46]:
# Download current results for each project
number_of_files_to_download = len(survey_ids)
count = 1
for survey_id in survey_ids:
    dl_link = f"{cfg.current_results_dl_url_prefix}{survey_id}"
    print(f'\nNow downloading file {count} of {number_of_files_to_download}')
    print(f'\nNow downloading SurveyResults for {dl_link}')
    driver.get(dl_link)
    count += 1

 2023-05-31 20:38:38,503 - DEBUG - POST http://localhost:50479/session/330a02588ec749e46ddb6411501982b8/url {"url": "https://data.studentedge.org/admin/survey/download?id=c133b54d-261f-44cc-b71d-b01201564af0"}



Now downloading file 1 of 147

Now downloading SurveyResults for https://data.studentedge.org/admin/survey/download?id=c133b54d-261f-44cc-b71d-b01201564af0


 2023-05-31 20:39:01,420 - DEBUG - http://localhost:50479 "POST /session/330a02588ec749e46ddb6411501982b8/url HTTP/1.1" 200 14
 2023-05-31 20:39:01,420 - DEBUG - Remote response: status=200 | data={"value":null} | headers=HTTPHeaderDict({'Content-Length': '14', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
 2023-05-31 20:39:01,420 - DEBUG - Finished Request
 2023-05-31 20:39:01,421 - DEBUG - POST http://localhost:50479/session/330a02588ec749e46ddb6411501982b8/url {"url": "https://data.studentedge.org/admin/survey/download?id=b8833dc5-e315-4e2f-a995-b013002b4db6"}



Now downloading file 2 of 147

Now downloading SurveyResults for https://data.studentedge.org/admin/survey/download?id=b8833dc5-e315-4e2f-a995-b013002b4db6


 2023-05-31 20:39:24,424 - DEBUG - http://localhost:50479 "POST /session/330a02588ec749e46ddb6411501982b8/url HTTP/1.1" 200 14
 2023-05-31 20:39:24,425 - DEBUG - Remote response: status=200 | data={"value":null} | headers=HTTPHeaderDict({'Content-Length': '14', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
 2023-05-31 20:39:24,425 - DEBUG - Finished Request
 2023-05-31 20:39:24,425 - DEBUG - POST http://localhost:50479/session/330a02588ec749e46ddb6411501982b8/url {"url": "https://data.studentedge.org/admin/survey/download?id=c41f2b22-44d5-49b3-84f9-b00700329155"}


...

 2023-05-31 21:40:26,269 - DEBUG - http://localhost:50479 "POST /session/330a02588ec749e46ddb6411501982b8/url HTTP/1.1" 200 14
 2023-05-31 21:40:26,269 - DEBUG - Remote response: status=200 | data={"value":null} | headers=HTTPHeaderDict({'Content-Length': '14', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
 2023-05-31 21:40:26,269 - DEBUG - Finished Request
 2023-05-31 21:40:26,270 - DEBUG - POST http://localhost:50479/session/330a02588ec749e46ddb6411501982b8/url {"url": "https://data.studentedge.org/admin/survey/download?id=3ef5e6b0-4c00-417f-83c1-af5f001ca572"}



Now downloading file 146 of 147

Now downloading SurveyResults for https://data.studentedge.org/admin/survey/download?id=3ef5e6b0-4c00-417f-83c1-af5f001ca572


 2023-05-31 21:40:49,532 - DEBUG - http://localhost:50479 "POST /session/330a02588ec749e46ddb6411501982b8/url HTTP/1.1" 200 14
 2023-05-31 21:40:49,533 - DEBUG - Remote response: status=200 | data={"value":null} | headers=HTTPHeaderDict({'Content-Length': '14', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
 2023-05-31 21:40:49,533 - DEBUG - Finished Request
 2023-05-31 21:40:49,534 - DEBUG - POST http://localhost:50479/session/330a02588ec749e46ddb6411501982b8/url {"url": "https://data.studentedge.org/admin/survey/download?id=be153b50-0230-4009-96a6-af3f003b48cd"}



Now downloading file 147 of 147

Now downloading SurveyResults for https://data.studentedge.org/admin/survey/download?id=be153b50-0230-4009-96a6-af3f003b48cd


 2023-05-31 21:41:14,424 - DEBUG - http://localhost:50479 "POST /session/330a02588ec749e46ddb6411501982b8/url HTTP/1.1" 200 14
 2023-05-31 21:41:14,424 - DEBUG - Remote response: status=200 | data={"value":null} | headers=HTTPHeaderDict({'Content-Length': '14', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
 2023-05-31 21:41:14,425 - DEBUG - Finished Request
